In [1]:
import warnings
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration # type: ignore
import os
import pygame # type: ignore
from gtts import gTTS # type: ignore
from time import sleep
import speech_recognition as sr # type: ignore
import tempfile
import datetime  # New import for date and time

# Suppress FutureWarning for tokenization spaces
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers.tokenization_utils_base")


pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
# Load the model and tokenizer globally to avoid reloading every time
model_name = "facebook/blenderbot-400M-distill"
tokenizer = BlenderbotTokenizer.from_pretrained(model_name)
model = BlenderbotForConditionalGeneration.from_pretrained(model_name)


In [ ]:
def capture_speech(mic_index):
    recognizer = sr.Recognizer()
    recognizer.energy_threshold = 300  # Lower energy threshold for faster detection
    mic = sr.Microphone(device_index=mic_index)
    
    with mic as source:
        print("Listening for your query...")
        recognizer.adjust_for_ambient_noise(source, duration=0.5)  # Optimized ambient noise detection
        
        try:
            # Listen with optimized timeout and phrase time limit
            audio = recognizer.listen(source, timeout=5, phrase_time_limit=6)  # Slightly reduced timeout

        except sr.WaitTimeoutError:
            print("Listening timed out, please speak again.")
            return None  # Return None if timeout occurs

    try:
        user_input = recognizer.recognize_google(audio)
        print(f"User said: {user_input}")  # Display user input
        return user_input
    except sr.UnknownValueError:
        print("Sorry, I could not understand the audio.")
        return None
    except sr.RequestError as e:
        print(f"Error with the Google Speech Recognition service; {e}")
        return None


In [ ]:
def handle_time_date_queries(user_input):
    if "time" in user_input:
        current_time = datetime.datetime.now().strftime("%I:%M %p")
        return f"The current time is {current_time}."
    elif "date" in user_input or "today" in user_input:
        current_date = datetime.datetime.now().strftime("%A, %B %d, %Y")
        return f"Today is {current_date}."
    else:
        return None


In [ ]:
def select_microphone():
    mic_list = sr.Microphone.list_microphone_names()
    return mic_list,
    index = 2
    # Removed print statement for microphone selection
    return index
print(mic_list,index)



NameError: name 'mic_list' is not defined

In [ ]:
def speak_response(text):
    tts = gTTS(text=text, lang='en')
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_audio_file:
        tts.save(temp_audio_file.name)
        temp_file_path = temp_audio_file.name
    pygame.mixer.init()
    try:
        pygame.mixer.music.load(temp_file_path)
        pygame.mixer.music.play()

        while pygame.mixer.music.get_busy():
            pygame.time.wait(50)  # Check every 50ms for faster termination

    except Exception as e:
        st.write(f"Error playing audio: {e}")
    finally:
        pygame.mixer.quit()
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)

In [ ]:
def generate_response(user_input):
    # Check if the user asked for the AI's name
    if "what is your name" in user_input.lower():
        return "I am Amigo, your all-weather conversational companion, just like the bike that you are riding on."
    
    # Handle time and date queries separately
    time_date_response = handle_time_date_queries(user_input)
    if time_date_response:
        return time_date_response

    # Check for factual or current event queries
    if any(keyword in user_input.lower() for keyword in [ "who is", "when is", "where is", "current", "today", "news","capital","Places"]):
        return "Thank you for your query, but that's out of my scope of training."

    # Tokenize user input
    inputs = tokenizer(user_input, return_tensors="pt", truncation=True, max_length=50)  # Reduced max length
    attention_mask = inputs['attention_mask']
    # Generate bot response with reduced max output length
    reply_ids = model.generate(inputs['input_ids'], max_length=60, pad_token_id=tokenizer.eos_token_id)  # Reduced response length
    
    # Decode response with clean-up for tokenization spaces
    bot_response = tokenizer.decode(reply_ids[0], skip_special_tokens=True,attention_mask=attention_mask, clean_up_tokenization_spaces=True)
    
    return bot_response


In [ ]:
def listen_for_wake_word(mic_index):
    recognizer = sr.Recognizer()
    mic = sr.Microphone(device_index=mic_index)
    
    print("Waiting for wake word 'Hey Amigo'...")
    while True:
        with mic as source:
            recognizer.adjust_for_ambient_noise(source, duration=0.5)  # Faster ambient noise adjustment
            audio = recognizer.listen(source, timeout=5)  # Timeout added to avoid long waits
            
        try:
            user_input = recognizer.recognize_google(audio)
            print(f"User said: {user_input}")
            if "hey amigo" in user_input.lower():
                print("Wake word detected!")  # Display wake word detected
                return True  # Wake word detected, start conversation
        except sr.UnknownValueError:
            continue
        except sr.RequestError as e:
            print(f"Error with the Google Speech Recognition service: {e}")
            continue


In [ ]:
def amigo_conversational_companion():
    mic_index = select_microphone()

    # Step 1: Wait for the wake word "Hey Amigo"
    if listen_for_wake_word(mic_index):
        # Step 2: After wake word, greet the user
        greeting = "Hi Hello Namaskara, I am Amigo. How can I help you today?"
        print(f"Amigo: {greeting}")
        speak_response(greeting)
        
        # Step 3: Enter conversation loop
        while True:
            user_input = capture_speech(mic_index)
            
            if user_input is None:
                continue
            
            # Check if the user wants to exit
            if "exit" in user_input.lower() or "goodbye" in user_input.lower():
                farewell_message = "Goodbye, have a nice day!"
                print(f"Amigo: {farewell_message}")
                speak_response(farewell_message)
                break  # Exit the loop
            
            # Generate bot response for normal conversation
            bot_response = generate_response(user_input)
            print(f"Amigo: {bot_response}")
            
            # Speak the bot's response
            speak_response(bot_response)


In [ ]:
if __name__ == "__main__":
    amigo_conversational_companion()


Waiting for wake word 'Hey Amigo'...


WaitTimeoutError: listening timed out while waiting for phrase to start